Data Ingestion

In [12]:
import os
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
    CharacterTextSplitter
)
from langchain_core.documents import Document
import pandas as pd
from typing import List, Dict, Any
print("Setup complete!")

Setup complete!


In [13]:
doc = Document(
    page_content="Hello, world!",
    metadata={
        "source": "example.txt",
        "page": 1,
        "author": "Aditya VVS",
        "date_created": "2025-01-06",
        "custom_field": "any_value"
    }
)
print("Document Structure")
print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")

Document Structure
Content: Hello, world!
Metadata: {'source': 'example.txt', 'page': 1, 'author': 'Aditya VVS', 'date_created': '2025-01-06', 'custom_field': 'any_value'}


## Text Files (.txt)

In [21]:
# Create a simple text file
import os
os.makedirs("data/text_files",exist_ok=True)

In [22]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python is a high-level, interpreted programming language known for its simplicity and readability. 
It was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity 
and allows developers to express concepts in fewer lines of code compared to many other languages.

Python supports multiple programming paradigms, including procedural, object-oriented, and 
functional programming. This flexibility makes it suitable for a wide range of applications, 
from small automation scripts to large-scale enterprise systems.

One of Python’s greatest strengths is its extensive standard library and third-party ecosystem. 
Libraries such as NumPy and Pandas are widely used for data analysis, while frameworks like 
Django and Flask are popular for web development. Python is also a leading language in the 
field of artificial intelligence and machine learning, with libraries such as TensorFlow, 
PyTorch, and scikit-learn.

Python is cross-platform and runs on Windows, macOS, and Linux. Its strong community support 
and open-source nature have contributed to its rapid adoption across industries such as finance, 
healthcare, education, and cloud computing.

Because of its ease of learning and powerful capabilities, Python is often recommended as a 
first programming language for beginners, while still being powerful enough for experienced 
developers and researchers.
""",
"data/text_files/python_intro_2.txt":"""
With Python, you can create a wide range of applications, from simple scripts to complex 
enterprise systems. Here are some examples of what you can build:

1. Automation: Automate repetitive tasks, such as data entry, file management, and system maintenance.

2. Web Development: Create dynamic websites and web applications using frameworks like Django or Flask.
"""
}
for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding='utf-8') as f:
        f.write(content)
print("Sample text files created!")

Sample text files created!


### TextLoader - Read a single txt file

In [23]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("data/text_files/python_intro.txt",encoding='utf-8')
loader  #loader type

In [24]:
documents = loader.load()
print(type(documents))
print(documents)

<class 'list'>
[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python is a high-level, interpreted programming language known for its simplicity and readability. \nIt was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity \nand allows developers to express concepts in fewer lines of code compared to many other languages.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and \nfunctional programming. This flexibility makes it suitable for a wide range of applications, \nfrom small automation scripts to large-scale enterprise systems.\n\nOne of Python’s greatest strengths is its extensive standard library and third-party ecosystem. \nLibraries such as NumPy and Pandas are widely used for data analysis, while frameworks like \nDjango and Flask are popular for web development. Python is also a leading language in the \nfield of artificial intelligence and machine learning, with libr

In [25]:
print(f"Loaded {len(documents)} documents")
print(f"First document content: {documents[0].page_content}")
print(f"First document metadata: {documents[0].metadata}")

Loaded 1 documents
First document content: Python is a high-level, interpreted programming language known for its simplicity and readability. 
It was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity 
and allows developers to express concepts in fewer lines of code compared to many other languages.

Python supports multiple programming paradigms, including procedural, object-oriented, and 
functional programming. This flexibility makes it suitable for a wide range of applications, 
from small automation scripts to large-scale enterprise systems.

One of Python’s greatest strengths is its extensive standard library and third-party ecosystem. 
Libraries such as NumPy and Pandas are widely used for data analysis, while frameworks like 
Django and Flask are popular for web development. Python is also a leading language in the 
field of artificial intelligence and machine learning, with libraries such as TensorFlow, 
PyTorch, and scikit-learn.

Python is

### DirectoryLoader - Read all text files from the Directory. Suitable for multiple text files

In [26]:
from langchain_community.document_loaders import DirectoryLoader
## load all text files from the directory
dir_loader = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # Pattern to match the files
    loader_cls=TextLoader, # Because there are multiple files, we need to specify which class to use for loading the text file
    loader_kwargs={"encoding":"utf-8"}, # Specify the encoding of the text file. Every text data needs to be encoded in utf-8
    show_progress = True # Show progress of the loading process
)
documents = dir_loader.load()

print(f"Loaded {len(documents)} documents")
for i,doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"Metadata: {doc.metadata}")
    print(f"document source: {doc.metadata['source']}")
    print(f"length of the document: {len(doc.page_content)}")
    print(f"Content: {doc.page_content[:100]}...")



100%|██████████| 2/2 [00:00<00:00, 422.07it/s]

Loaded 2 documents

Document 1:
Metadata: {'source': 'data\\text_files\\python_intro.txt'}
document source: data\text_files\python_intro.txt
length of the document: 1399
Content: Python is a high-level, interpreted programming language known for its simplicity and readability. 
...

Document 2:
Metadata: {'source': 'data\\text_files\\python_intro_2.txt'}
document source: data\text_files\python_intro_2.txt
length of the document: 366
Content: 
With Python, you can create a wide range of applications, from simple scripts to complex 
enterpris...


Text Splitting Techniques

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter, TokenTextSplitter
print(documents)
print(len(documents))

[Document(metadata={'source': 'data\\text_files\\python_intro.txt'}, page_content='Python is a high-level, interpreted programming language known for its simplicity and readability. \nIt was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity \nand allows developers to express concepts in fewer lines of code compared to many other languages.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and \nfunctional programming. This flexibility makes it suitable for a wide range of applications, \nfrom small automation scripts to large-scale enterprise systems.\n\nOne of Python’s greatest strengths is its extensive standard library and third-party ecosystem. \nLibraries such as NumPy and Pandas are widely used for data analysis, while frameworks like \nDjango and Flask are popular for web development. Python is also a leading language in the \nfield of artificial intelligence and machine learning, with libraries such as

In [29]:
### Method 1 - CharacterTextSplitter
text = documents[0].page_content
text

'Python is a high-level, interpreted programming language known for its simplicity and readability. \nIt was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity \nand allows developers to express concepts in fewer lines of code compared to many other languages.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and \nfunctional programming. This flexibility makes it suitable for a wide range of applications, \nfrom small automation scripts to large-scale enterprise systems.\n\nOne of Python’s greatest strengths is its extensive standard library and third-party ecosystem. \nLibraries such as NumPy and Pandas are widely used for data analysis, while frameworks like \nDjango and Flask are popular for web development. Python is also a leading language in the \nfield of artificial intelligence and machine learning, with libraries such as TensorFlow, \nPyTorch, and scikit-learn.\n\nPython is cross-platform and runs on 

In [30]:
# Method 1 - Character-based splitting
print("CHARACTER TEXT SPLITTER")
char_splitter = CharacterTextSplitter(
    separator="\n", # Split the text by new line
    chunk_size=100, # Max characters per chunk 
    chunk_overlap=20, # Overlap between chunks; last 20 characters of chunk 1 can be present in the beginning 20 characters of chunk 2
    length_function=len, # How to measure the chunk size
)
char_chunks = char_splitter.split_text(text)
print(f"Total chunks = {len(char_chunks)}")
print(char_chunks)
print("First chunk: ",char_chunks[0])




CHARACTER TEXT SPLITTER
Total chunks = 17
['Python is a high-level, interpreted programming language known for its simplicity and readability.', 'It was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity', 'and allows developers to express concepts in fewer lines of code compared to many other languages.', 'Python supports multiple programming paradigms, including procedural, object-oriented, and', 'functional programming. This flexibility makes it suitable for a wide range of applications,', 'from small automation scripts to large-scale enterprise systems.', 'One of Python’s greatest strengths is its extensive standard library and third-party ecosystem.', 'Libraries such as NumPy and Pandas are widely used for data analysis, while frameworks like', 'Django and Flask are popular for web development. Python is also a leading language in the', 'field of artificial intelligence and machine learning, with libraries such as TensorFlow,', 'PyTorch, and sci

In [33]:
# Method 2 - RecursiveCharacterTextSplitter
print("RECURSIVE CHARACTER TEXT SPLITTER")
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"], # Split the text by new line, space, and empty string
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)
recursive_chunks = recursive_splitter.split_text(text)
print(f"Total chunks = {len(recursive_chunks)}")
print(recursive_chunks)
print("First chunk: ",recursive_chunks[0])
print("Second chunk: ",recursive_chunks[1])


RECURSIVE CHARACTER TEXT SPLITTER
Total chunks = 17
['Python is a high-level, interpreted programming language known for its simplicity and readability.', 'It was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity', 'and allows developers to express concepts in fewer lines of code compared to many other languages.', 'Python supports multiple programming paradigms, including procedural, object-oriented, and', 'functional programming. This flexibility makes it suitable for a wide range of applications,', 'from small automation scripts to large-scale enterprise systems.', 'One of Python’s greatest strengths is its extensive standard library and third-party ecosystem.', 'Libraries such as NumPy and Pandas are widely used for data analysis, while frameworks like', 'Django and Flask are popular for web development. Python is also a leading language in the', 'field of artificial intelligence and machine learning, with libraries such as TensorFlow,', 'PyTorc

In [34]:
# Create text without natural breaks
sample_text = "This is a sample text. It is a test of the recursive character text splitter. It is a test of the recursive character text splitter. It is a test of the recursive character text splitter."
splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=80,
    chunk_overlap=20,
    length_function=len
)
chunks = splitter.split_text(sample_text)
print(f"Total chunks = {len(chunks)}")

for i in range(len(chunks)-1):
    print(f"Chunk {i+1}: {chunks[i]}")
    print(f"Chunk {i+2}: {chunks[i+1]}")

    print()


Total chunks = 3
Chunk 1: This is a sample text. It is a test of the recursive character text splitter. It
Chunk 2: text splitter. It is a test of the recursive character text splitter. It is a

Chunk 2: text splitter. It is a test of the recursive character text splitter. It is a
Chunk 3: splitter. It is a test of the recursive character text splitter.



In [37]:
# Method 3 - Token Based Splitting
print("TOKEN TEXT SPLITTER")
token_splitter = TokenTextSplitter(
    chunk_size=50, # Size in tokens, not characters
    chunk_overlap=10,
    length_function=len
)
token_chunks = token_splitter.split_text(text)
print(f"Created {len(token_chunks)} chunks")
print(f"First chunk: {token_chunks[0]}")
print(f"First chunk length: {len(token_chunks[0])}")

TOKEN TEXT SPLITTER
Created 8 chunks
First chunk: Python is a high-level, interpreted programming language known for its simplicity and readability. 
It was created by Guido van Rossum and first released in 1991. Python emphasizes code clarity 
and allows developers to express concepts in fewer lines
First chunk length: 251
